<a href="https://colab.research.google.com/github/sanath8107/strategies/blob/main/rsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Relative Strength Index**
\
The relative strength index (RSI) is a momentum indicator used in technical analysis that measures the magnitude of recent price changes to evaluate overbought or oversold conditions in the price of a stock or other asset. The RSI is displayed as an oscillator (a line graph that moves between two extremes) and can have a reading from 0 to 100.



*  An asset is usually considered overbought when the RSI is above 70% and oversold when it is below 30%.
*   Therefore we buy when RSI < 30 and sell when RSI > 70

